## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np

# Import the API key.
from config import g_key

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Jamestown,42.0970,-79.2353,51.39,91,90,5.01,US,overcast clouds,2021-11-01 04:26:53
1,1,Cherskiy,68.7500,161.3000,-2.74,99,100,5.53,RU,overcast clouds,2021-11-01 04:19:52
2,2,Saskylakh,71.9167,114.0833,3.78,97,98,5.61,RU,overcast clouds,2021-11-01 04:29:38
3,3,Rikitea,-23.1203,-134.9692,72.99,76,100,16.78,PF,overcast clouds,2021-11-01 04:27:35
4,4,Provideniya,64.3833,-173.3000,17.71,62,40,11.18,RU,scattered clouds,2021-11-01 04:18:49


In [8]:
# Verify lat and lng are float64
city_data_df.dtypes

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Country                 object
Current Description     object
Date                    object
dtype: object

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) ]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
7,7,Butaritari,3.0707,172.7902,82.62,73,6,12.24,KI,light rain,2021-11-01 04:15:20
9,9,Tabuk,17.4189,121.4443,82.85,67,57,4.72,PH,broken clouds,2021-11-01 04:29:40
14,14,Boa Vista,2.8197,-60.6733,82.38,78,40,6.91,BR,scattered clouds,2021-11-01 04:29:44
19,19,Aden,12.7794,45.0367,82.60,69,10,19.33,YE,clear sky,2021-11-01 04:29:46
20,20,Beirut,33.8889,35.4944,79.54,39,75,25.32,LB,broken clouds,2021-11-01 04:29:47
21,21,Urucara,-2.5364,-57.7600,75.16,88,77,2.19,BR,broken clouds,2021-11-01 04:29:48
25,25,Port Hedland,-20.3167,118.5667,80.65,61,90,11.50,AU,overcast clouds,2021-11-01 04:29:38
28,28,Atuona,-9.8000,-139.0333,78.84,73,0,18.50,PF,clear sky,2021-11-01 04:27:26
33,33,Nieuw Amsterdam,5.8833,-55.0833,78.03,90,57,7.34,SR,broken clouds,2021-11-01 04:29:55
35,35,Luanda,-8.8368,13.2343,75.20,78,75,5.75,AO,broken clouds,2021-11-01 04:29:56


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# Verify hotel dataframe has hotels with city information
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
7,Butaritari,KI,light rain,82.62,3.0707,172.7902,Isles Sunset Lodge
9,Tabuk,PH,broken clouds,82.85,17.4189,121.4443,Golden Berries Hotel and Convention
14,Boa Vista,BR,scattered clouds,82.38,2.8197,-60.6733,Aipana Plaza Hotel
19,Aden,YE,clear sky,82.60,12.7794,45.0367,Sama Emirate Hotel
20,Beirut,LB,broken clouds,79.54,33.8889,35.4944,"Crowne Plaza Beirut, an IHG Hotel"


In [17]:
# Info box data
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Save map
# plt.savefig("data\WeatherPy_vacation_map.png")

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [20]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index=False)